In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
import os

Using TensorFlow backend.


In [2]:
# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)
    
    
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    # Regression
    return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [3]:
path = "./data/"
save_path = "./dnn/"

In [4]:
filename_read = os.path.join(path,"Train_Solar.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

In [5]:
df.head(1)

,datetime,date,time,I,T,UV,WS,RH,P
0,1/1/2017 6:00,1/1/2017,6:00:00,0.0,24.6,0.0,1.2,59.8,0.0


In [6]:
datetime_sr = df['datetime']
date_sr = df['date']
time_sr = df['time']

In [7]:
df.drop('datetime',1,inplace=True)
df.drop('date',1,inplace=True)
df.drop('time',1,inplace=True)
print(df.head(1))

     I     T   UV   WS    RH    P
0  0.0  24.6  0.0  1.2  59.8  0.0


In [8]:
missing_median(df, 'I')
missing_median(df, 'T')
missing_median(df, 'UV')
missing_median(df, 'WS')
missing_median(df, 'RH')
missing_median(df, 'P')

In [9]:
x,y = to_xy(df,"P")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [10]:
print(type(x))
print(x.shape)
print(type(y))
print(y.shape)

<class 'numpy.ndarray'>
(131586, 5)
<class 'numpy.ndarray'>
(131586, 1)


In [11]:
print(x[:5])

[[ 0.    24.6    0.     1.2   59.8  ]
 [ 0.    24.5    0.     1.4   60.   ]
 [ 0.    24.5    0.     1.9   60.2  ]
 [ 0.    24.5    0.     1.3   60.3  ]
 [ 0.    24.6    0.     1.4   60.125]]


In [12]:
print(y[:5])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [13]:
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output

In [14]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [15]:
model.fit(x,y,verbose=0,epochs=100)

In [16]:
# Predict
pred = model.predict(x)

In [17]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Before save score (RMSE): {score}")

Before save score (RMSE): 0.4765164852142334


In [18]:
# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(path,"R1_model.json"), "w") as json_file:
    json_file.write(model_json)

In [19]:
# save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(path,"R1_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

In [20]:
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(path,"R1_model.h5"))

In [21]:
from keras.models import load_model
model2 = load_model(os.path.join(path,"R1_model.h5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 0.4765164852142334
